# Try to Recreate the Sound Module

Our initial attempt will be to recreate moomoo.dat using the syn6.o binary file that was released along with the sources and a reverse-engineered source of the first few bytes in moomoo.dat that are a jumpt table of sorts added in later.

Near the top of `yak.s` we have a bunch of 'equates' that provide sound functions used by the rest of the game:


```
INIT_SOUND    EQU  $4040  ;jump table for the SFX/Tunes module
NT_VBL        EQU  $4046
PT_MOD_INIT   EQU  $404c
START_MOD     EQU  $4052
STOP_MOD      EQU  $4058
PLAYFX2       EQU  $405e
CHANGE_VOLUME EQU  $4064
SET_VOLUME    EQU  $406a
NOFADE        EQU  $4070
FADEUP        EQU  $4076
FADEDOWN      EQU  $407c
ENABLE_FX     EQU  $4082
DISABLE_FX    EQU  $4088
CHANGEFX      EQU  $409a  ;new in syn6
HALT_DSP      EQU  $408e
RESUME_DSP    EQU  $4094
intmask       EQU  $40a0
```


For example, to play a tune we use a bunch of them like so. `START_MOD` at the very end starts playing the selected `mod` file. `mod` is the music file format the Tempest tunes are stored in.
    
```
; *******************************************************************
; Play a tune
; *******************************************************************
playtune:
  jsr STOP_MOD
  lsl #2,d0
  lea modbase,a0
  move.l 0(a0,d0.w),a0  ;get tune base
  jsr PT_MOD_INIT
  move.b vols,d0
  and.l #$ff,d0
  clr d1
  jsr SET_VOLUME
  move.l d0,vset
  jsr NOFADE
  jmp START_MOD
```

The source code for these functions isn't available directly. Instead we have a table of binary data in the file `moomoo.dat` which is gets loaded to address `0x40ab`. This is what the first bit of `moomoo.dat` looks like:


In [287]:
!head -n 40 moomoo.dat

This file is where our `START_MOD`, `STOP_MOD`, & co. functions live. The source code for them isn't available and it is likely that it wasn't available to Jeff Minter either. This is because the bulk of the file is made up of sound synthesis binary provided to him by Imagitec Designs. We have this binary, it's called `syn6.o`.

So let's recreate the likely sequence of steps used to create `moomoo.dat`:
* Create a source file with our jump table in it.
* Link it with then `syn6.o` binary.
* Hope that we now have something resembling the contents of `moomoo.dat`.



First, let's convert `moomoo.dat` to a binary file. This will enable us to compare like with like once we have a binary output of our own:

In [234]:
flatten = lambda l: [e for sublist in l for e in sublist]
def long_to_bytes(l):
    bytes = []
    for i in range(1,8,2):
        bytes += [int(l[i:i+2],16)]
    return bytes

dat_lines = open("moomoo.dat",'r').readlines()[10:554]
dat = [d.replace('\tDC.L\t','') for d in dat_lines]
dat = flatten([s.split(',') for s in dat if s])
dat_bytes = flatten([long_to_bytes(s) for s in dat])

binary = open("moomoo.bin", 'wb')
binary.write(bytes(dat_bytes))
binary.close()

Take a look at our moomoo.bin file, the binary conversion of moomoo.dat.

In [235]:
!dd if=moomoo.bin|xxd


16+1 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 3.1896e-05 s, 273 MB/s
00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A.N...B^N...
00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N...E.N...F.N.
00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN...D.N...EF
00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E.N...E.N...
00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N...FNN...F`N.
00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N...F`N...Dp
00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 4240  ..Nu?.......@.B@
00000070: 1039 0000 40a0 33c0 00f0 00e0 301f 4e75  .9..@.3.....0.Nu
00000080: 436f 7079 7269 6768 7420 2831 3939 3329  Copyright (1993)
00000090: 2049 6d61 6769 7465 6320 4465 7369 676e   Imagitec Design
000000a0: 2c20 496e 6320 48e7 fffe 23fc 0000 0000  , Inc H...#.....
000000b0: 00f1 a114 23fc 0000 0000 00f1 a100 227c  ....#........."|
000000c0: 00f1 b000 207c 0000 5ed4 2008 223c 0000  .... |..^. ."<..
000000d0: 6238 9280 e48

## Interlude - Get our Build Tools

We have two tools to allow us to build and link ASM68K source files the way it would have been done back in the day. The first is 'rmac', a successor to the popular `madmac` assembler used by Minter:

In [238]:
import os
!rm -rf rmac
!git clone https://github.com/mwenge/rmac.git
os.chdir('rmac')
!make -j5
os.chdir('..')

Cloning into 'rmac'...
remote: Enumerating objects: 2050, done.
remote: Counting objects: 100% (2050/2050), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2050 (delta 1641), reused 2050 (delta 1641), pack-reused 0
Receiving objects: 100% (2050/2050), 740.86 KiB | 2.65 MiB/s, done.
Resolving deltas: 100% (1641/1641), done.
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c kwgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c debug.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56k.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c eagen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o dsp56kgen dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o 68kgen 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o kwgen kwgen.c
gcc -std=c99 -D

listing.c: In function ‘ship_ln’:
listing.c:234:39: warning: ‘%-20s’ directive writing 20 or more bytes into a region of size between 1 and 984 []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
  234 |                                 "%-40s%-20s Page %-4d    %s %s        RMAC %01i.%01i.%02i (%s)",
      |                                       ^~~~~
In file included from /usr/include/stdio.h:894,
                 from rmac.h:12,
                 from listing.h:13,
                 from listing.c:16:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:38:10: note: ‘__builtin___sprintf_chk’ output 107 or more bytes (assuming 1128) into a destination of size 1024
   38 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   39 |                                   __glibc_objsize (__s), __fmt,
      |                                   ~~~~~~~~~~~~~~~~~~~~

Next we build `rln`, the successor to `aln` the linker-de-jour for Jaguar:

In [5]:
import os
!rm -rf rln
!git clone http://shamusworld.gotdns.org/git/rln
os.chdir('rln')
!make -j5
os.chdir('..')

Cloning into 'rln'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 216 (delta 107), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (216/216), 160.16 KiB | 377.00 KiB/s, done.
Resolving deltas: 100% (107/107), done.
/bin/rm -f rln.o
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -c rln.c
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -o rln rln.o 


## We're Back

Comparing `moomoo.bin` and `syn6.o` doesn't give us much initial grounds for hope. At first glance they look very different:

In [289]:
!diff -y -W 120 <(dd if=moomoo.bin|xxd) <(dd if=syn6.o|xxd)

16+1 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 2.5567e-05 s, 340 MB/s
48+1 records in
48+1 records out
24640 bytes (25 kB, 24 KiB) copied, 4.2409e-05 s, 581 MB/s
00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A   |	00000000: 601a 0000 2178 0000 0000 0000 0000 0000  `...!
00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N..   |	00000010: 1d34 0000 0000 0000 0000 0000 436f 7079  .4...
00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN   |	00000020: 7269 6768 7420 2831 3939 3329 2049 6d61  right
00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E   |	00000030: 6769 7465 6320 4465 7369 676e 2c20 496e  gitec
00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N..   |	00000040: 6320 48e7 fffe 23fc 0000 0000 00f1 a114  c H..
00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N   |	00000050: 23fc 0000 0000 00f1 a100 227c 0000 0000  #....
00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 4240  ..Nu?   |	00000060: 207c 0000 0000 2008 223c 0000 0000 9

However, closer inspection reveals that the files are in fact broadly the same. THey just have different header material. So if we start the comparison at slightly different positions in each file we can see where the main body of `syn6.o` is contained in `moomoo.bin`:

In [291]:
!diff -y -W 120 <(dd ibs=1 skip=128 if=moomoo.bin|xxd) <(dd ibs=1 skip=28 if=syn6.o|xxd)

8568+0 records in
16+1 records out
8568 bytes (8.6 kB, 8.4 KiB) copied, 0.00446836 s, 1.9 MB/s
24612+0 records in
48+1 records out
24612 bytes (25 kB, 24 KiB) copied, 0.00832187 s, 3.0 MB/s
00000000: 436f 7079 7269 6768 7420 2831 3939 3329  Copyr	00000000: 436f 7079 7269 6768 7420 2831 3939 3329  Copyr
00000010: 2049 6d61 6769 7465 6320 4465 7369 676e   Imag	00000010: 2049 6d61 6769 7465 6320 4465 7369 676e   Imag
00000020: 2c20 496e 6320 48e7 fffe 23fc 0000 0000  , Inc	00000020: 2c20 496e 6320 48e7 fffe 23fc 0000 0000  , Inc
00000030: 00f1 a114 23fc 0000 0000 00f1 a100 227c  ....#	00000030: 00f1 a114 23fc 0000 0000 00f1 a100 227c  ....#
00000040: 00f1 b000 207c 0000 5ed4 2008 223c 0000  ....    |	00000040: 0000 0000 207c 0000 0000 2008 223c 0000  .... 
00000050: 6238 9280 e489 22d8 51c9 fffc 207c 00f1  b8...   |	00000050: 0000 9280 e489 22d8 51c9 fffc 207c 00f1  .....
00000060: b800 4eb9 0000 4174 4eb9 0000 4174 4eb9  ..N..   |	00000060: b800 4eb9 0000 00b4 4eb9 0000 00b4 4eb9  ..N..


It turns out that the first 28 bytes of `syn6.o` are an ALCYON object file header.

In [293]:
syn6 = open("syn6.o",'rb')
syn6.seek(0)
print("Magic number",syn6.read(2).hex(), "Alcyon-Format Object Module")
text_size = syn6.read(4)
print("Text Size",text_size.hex())
data_size = syn6.read(4)
print("Data Size",data_size.hex())
print("BSS Size",syn6.read(4).hex())
sym_size = syn6.read(4)
print("Symbol Size",sym_size.hex())
print("Reserved (must be zero)",syn6.read(10).hex())


Magic number 601a Alcyon-Format Object Module
Text Size 00002178
Data Size 00000000
BSS Size 00000000
Symbol Size 00001d34
Reserved (must be zero) 00000000000000000000


The first 128 bytes of `moomoo.bin` on the other hand are a chunk of `jmp` statements and a couple of small routines at the end . For example:
```
4ef9 0000 41ba
```
is the assembled machine code for the statement:
```
jmp (0x000041ba).l
```

In [294]:
!dd ibs=1 count=128 if=moomoo.bin|xxd

128+0 records in
0+1 records out
128 bytes copied, 5.1824e-05 s, 2.5 MB/s
00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A.N...B^N...
00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N...E.N...F.N.
00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN...D.N...EF
00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E.N...E.N...
00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N...FNN...F`N.
00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N...F`N...Dp
00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 4240  ..Nu?.......@.B@
00000070: 1039 0000 40a0 33c0 00f0 00e0 301f 4e75  .9..@.3.....0.Nu


When we disassemble this machine code we get a little source file we call `moomoo_header.s`. This is a jump table of sorts and a couple of extra routines with the addresses 0x40a2 and 0x40a4 (which will be important later). 


When it comes to naming things in this disassembly we've used the names as given in `yak.s`:


```
INIT_SOUND    EQU  $4040  ;jump table for the SFX/Tunes module
NT_VBL        EQU  $4046
PT_MOD_INIT   EQU  $404c
START_MOD     EQU  $4052
STOP_MOD      EQU  $4058
PLAYFX2       EQU  $405e
CHANGE_VOLUME EQU  $4064
SET_VOLUME    EQU  $406a
NOFADE        EQU  $4070
FADEUP        EQU  $4076
FADEDOWN      EQU  $407c
ENABLE_FX     EQU  $4082
DISABLE_FX    EQU  $4088
CHANGEFX      EQU  $409a  ;new in syn6
HALT_DSP      EQU  $408e
RESUME_DSP    EQU  $4094
intmask       EQU  $40a0
```

So for example we know to call our first routine INIT_SOUND because it appears first above (at address `$4040`). So this:

```
4ef9 0000 41ba
```
becomes:
```
jmp (0x000041ba).l
```
which becomes:
```
INIT_SOUND:
    jmp (0x000041ba).l
```

We abitrarily give `0x000041ba` the name `init_sound` as it is (presumably) jumping to the routine in `syn6.o` that inits the sound module.

We repeat this process for all the other jump instructions and end up with a file called `moomoo_header.s` like so:



In [283]:
!cat moomoo_header.s

; Header for moomoo.s
;.org $4040
.include "jaguar.inc"

init_sound    EQU $000041ba
nt_vbl        EQU $0000425e
pt_mod_init   EQU $000041d6
start_mod     EQU $000045e2
stop_mod      EQU $00004606
playfx2       EQU $00004370
change_volume EQU $000044ea
set_volume    EQU $00004546
nofade        EQU $00004598
fadeup        EQU $000045b2
fadedown      EQU $000045ca
enable_fx     EQU $0000464e
disable_fx    EQU $00004660
resume_dsp    EQU $00004470

INIT_SOUND:
                jmp     (init_sound).l
NT_VBL:
                jmp     (nt_vbl).l
PT_MOD_INIT:
                jmp     (pt_mod_init).l
START_MOD:
                jmp     (start_mod).l
STOP_MOD:
                jmp     (stop_mod).l
PLAYFX2:
                jmp     (playfx2).l
CHANGE_VOLUME:
                jmp     (change_volume).l
SET_VOLUME:
                jmp     (set_volume).l
NOFADE:
                jmp     (nofade).l
FADEUP:
                jmp     (fadeup).l
FADEDOWN:
                jmp     (fadedown).l
ENABLE_FX:
          

So what we can do is build it as a separate file and then try linking it with `syn6.o` to see if we get something resembling `moomoo.bin`.

So let's build `moomoo_header.s` and create an absolute object file.

In [240]:
!./rmac/rmac -fa -i moomoo_header.s -o moomoo_header.o
!dd if=moomoo_header.o|xxd

6+1 records in
6+1 records out
3441 bytes (3.4 kB, 3.4 KiB) copied, 4.531e-05 s, 75.9 MB/s
00000000: 0000 0107 0000 0080 0000 0000 0000 0000  ................
00000010: 0000 0804 0000 0000 0000 0010 0000 0000  ................
00000020: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A.N...B^N...
00000030: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N...E.N...F.N.
00000040: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN...D.N...EF
00000050: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E.N...E.N...
00000060: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N...FNN...F`N.
00000070: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N...F`N...Dp
00000080: 0000 4e75 3f00 08f9 0003 0000 0060 4240  ..Nu?........`B@
00000090: 1039 0000 0060 33c0 00f0 00e0 301f 4e75  .9...`3.....0.Nu
000000a0: 0000 006a 0000 0440 0000 0072 0000 0440  ...j...@...r...@
000000b0: 0000 0004 0200 0000 00f0 0000 0000 0009  ................
000000c0: 0200 0000 00f0 0000 0000 0011 0200 0000  ................
000000d0: 00f0 0002 0000 

Now link moomoo_header and syn6.o together. The result should match the contents of moomoo.dat/moomo.bin. The start address of 0x4040 we specify to `rln` is determined by the references to the jump table given in `yak.s`:

```
INIT_SOUND    EQU  $4040  ;jump table for the SFX/Tunes module

```

In [241]:
!./rln/rln -z -u -v -a 4040 xd xd -e moomoo_header.o syn6.o -o moomoo.abs

      _
 _ __| |_ ___
| '__| | '_  \
| |  | | | | |
|_|  |_|_| |_|

Renamed Linker for Atari Jaguar
Copyright (c) 199x Allan K. Pratt, 2014-2021 Reboot & Friends
V1.7.4 Jun 21 2024 (OSX/Linux)

DoFile() : `moomoo_header.o' NORMAL
DoFile() : `syn6.o' NORMAL
Output file is moomoo.abs
Read file moomoo_header.o
Read file syn6.o
UNRESOLVED SYMBOLS
	ENABLETI (syn6.o)
	DISABLET (syn6.o)
RELOCSIZE :: 16  Records = 2
RELOCSIZE :: 184  Records = 23
Absolute linking (COF)

+---------+----------+----------+----------+
| Segment |     TEXT |     DATA |      BSS |
| Sizes   |----------+----------+----------|
| (Hex)   |     21F8 |        0 |        0 |
+---------+----------+----------+----------+

Link complete.


In [242]:
!diff -y -W 120 <(dd ibs=1 skip=168 if=moomoo.abs|xxd) <(dd if=moomoo.bin|xxd)|grep '  |	'

16+1 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 4.0674e-05 s, 214 MB/s
8696+0 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 0.00212317 s, 4.1 MB/s
000005b0: 0000 0000 49fa 18aa 50ec 0027 397c 0001  ....I   |	000005b0: 0000 40a4 49fa 18aa 50ec 0027 397c 0001  ..@.I
000005c0: 0014 285f 4e75 48e7 0118 4eb9 0000 0000  ..(_N   |	000005c0: 0014 285f 4e75 48e7 0118 4eb9 0000 40a2  ..(_N
000006a0: be7c 000f 6606 4eb9 0000 0000 4680 23c0  .|..f   |	000006a0: be7c 000f 6606 4eb9 0000 40a2 4680 23c0  .|..f
000006d0: 6706 4eb9 0000 0000 4cdf 1280 4e75 436f  g.N..   |	000006d0: 6706 4eb9 0000 40a4 4cdf 1280 4e75 436f  g.N..


They almost match. `rln` creates a 168 byte file header (which we can't seem to turn off) so we skip past that and compare our moomoo.abs file with moomoo.bin and have four locations where the bytes are different. In each case this is a `jsr` instruction that references a location of 0x40a2 or 0x40a4 in `moomoo.bin` but which is given as all zeroes in `syn6.o`.  THese 2 addresses are routines in our `moomoo_header.s` file, so were added by Minter (presumably) and then patched into the `syn6.o` binary after maybe?

So there must be some sort of binary patching that occurred here? Or else `syn6.o` was rebuilt using the sources and moomoo.dat generated with the result?

Anyway let's patch in the addresses ourselves:

In [282]:
moo_bytes = bytearray(open("moomoo.abs",'rb').read())
# for some reason, replacing more than one byte at a time
# results in a trailing 00
moo_bytes[168+0x5b2] = 0x40
moo_bytes[168+0x5b3] = 0xa4
moo_bytes[168+0x5ce] = 0x40
moo_bytes[168+0x5cf] = 0xa2
moo_bytes[168+0x6aa] = 0x40
moo_bytes[168+0x6ab] = 0xa2
moo_bytes[168+0x6d6] = 0x40
moo_bytes[168+0x6d7] = 0xa4

# Write out our patched file and skip the first 168 bytes of header.
open("moomoo_match.bin",'wb').write(moo_bytes[168:])

8696

Now, do we have a match?

In [281]:
#!diff -y -W 120 <(dd if=moomoo_match.bin|xxd) <(dd if=moomoo.bin|xxd)
!diff -s moomoo_match.bin moomoo.bin

Files moomoo_match.bin and moomoo.bin are identical


We do! We've successfully recreated `moomoo.bin`.

## Try recreating using a single source file

In [165]:
!./rmac/rmac -help



 _ __ _ __ ___   __ _  ___ 
| '__| '_ ` _ \ / _` |/ __|
| |  | | | | | | (_| | (__ 
|_|  |_| |_| |_|\__,_|\___|

Renamed Macro Assembler
Copyright (C) 199x Landon Dyer, 2011-2021 Reboot and Friends
V2.1.1 Jun 21 2024 (Linux/OSX)

Usage:
    ./rmac/rmac [options] srcfile

Options:
  -? or -h          Display usage information
  -dsymbol[=value]  Define symbol (with optional value, default=0)
  -e[errorfile]     Send error messages to file, not stdout
  -f[format]        Output object file format
                    a: ALCYON
                    b: BSD (use this for Jaguar)
                    e: ELF
                    p: P56 (use this for DSP56001 only)
                    l: LOD (use this for DSP56001 only)
                    x: com/exe/xex (Atari 800)
                    r: absolute address
  -i[path]          Directory to search for include files
  -l[filename]      Create an output listing file
  -l*[filename]     Create an output listing file without pagination
  -m[cpu]        

In [193]:
#!./rmac/rmac -fa -i moomoo_header.s -o moomoo_header.o
#!./rmac/rmac -s -fb -i moomoo.s -o moomoo.o
!./rmac/rmac -s ~oall -n -ps -i moomoo.s -o moomoo.o


Legacy mode OFF
moomoo.s 114: Warning: unoptimized short branch
moomoo.s 1052: Warning: unoptimized short branch


In [196]:
!dd if=moomoo.o|xxd

28+1 records in
28+1 records out
14458 bytes (14 kB, 14 KiB) copied, 3.4994e-05 s, 413 MB/s
00000000: 601a 0000 21f0 0000 0000 0000 0000 0000  `...!...........
00000010: 1650 0000 0000 0000 0000 0000 4ef9 0000  .P..........N...
00000020: 0174 4ef9 0000 0218 4ef9 0000 0190 4ef9  .tN.....N.....N.
00000030: 0000 059c 4ef9 0000 05c0 4ef9 0000 032a  ....N.....N....*
00000040: 4ef9 0000 0500 4ef9 0000 0552 4ef9 0000  N.....N....RN...
00000050: 056c 4ef9 0000 0584 4ef9 0000 0608 4ef9  .lN.....N.....N.
00000060: 0000 061a 4ef9 0000 061a 4ef9 0000 061a  ....N.....N.....
00000070: 4ef9 0000 06b4 0000 4e75 3f00 08f9 0003  N.......Nu?.....
00000080: 0000 005a 4240 1039 0000 005a 33c0 00f0  ...ZB@.9...Z3...
00000090: 00e0 301f 4e75 436f 7079 7269 6768 7420  ..0.NuCopyright 
000000a0: 2831 3939 3329 2049 6d61 6769 7465 6320  (1993) Imagitec 
000000b0: 4465 7369 676e 2c20 496e 6320 48e7 fffe  Design, Inc H...
000000c0: 23fc 0000 0000 00f1 a114 23fc 0000 0000  #.........#.....
000000d0: 00f1 a100 227c

In [185]:
!./rln/rln -z -u -v -a 4040 xd xd -e moomoo.o -o moomoo_src.abs

      _
 _ __| |_ ___
| '__| | '_  \
| |  | | | | |
|_|  |_|_| |_|

Renamed Linker for Atari Jaguar
Copyright (c) 199x Allan K. Pratt, 2014-2021 Reboot & Friends
V1.7.4 Jun 21 2024 (OSX/Linux)

DoFile() : `moomoo.o' NORMAL
Output file is moomoo_src.abs
Read file moomoo.o
Alcyon object file moomoo.o truncated: Header wants 23116B, file is 14458B
Link aborted.


In [181]:
!diff -y -W 120 <(dd ibs=1 skip=168 if=moomoo_src.abs|xxd) <(dd if=moomoo.bin|xxd)

16+1 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 2.7752e-05 s, 313 MB/s
8672+0 records in
16+1 records out
8672 bytes (8.7 kB, 8.5 KiB) copied, 0.00204727 s, 4.2 MB/s
00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A	00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A
00000010: 41d6 4ef9 0000 45d6 4ef9 0000 45fa 4ef9  A.N..   |	00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N..
00000020: 0000 436c 4ef9 0000 453e 4ef9 0000 458c  ..ClN   |	00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN
00000030: 4ef9 0000 45a6 4ef9 0000 45be 4ef9 0000  N...E   |	00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E
00000040: 4642 4ef9 0000 4654 4ef9 0000 4654 4ef9  FBN..   |	00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N..
00000050: 0000 4654 4ef9 0000 46ee 0000 4e75 3f00  ..FTN   |	00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N
00000060: 08f9 0003 0000 40a4 4240 1039 0000 40a4  .....   |	00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 42

Parse out the `syn6.o` binary.

In [50]:
syn6 = open("moomoo.bin",'rb')
syn6.seek(0)
print("Magic number",syn6.read(2).hex(), "Alcyon-Format Object Module")
text_size = syn6.read(4)
print("Text Size",text_size.hex())
data_size = syn6.read(4)
print("Data Size",data_size.hex())
print("BSS Size",syn6.read(4).hex())
sym_size = syn6.read(4)
print("Symbol Size",sym_size.hex())
print("Reserved (must be zero)",syn6.read(10).hex())
text_data = syn6.read(0x2178)
print("Symbol Table")
sym_size_int = int.from_bytes(sym_size, "big")
for i in range(0,int(sym_size_int/14)):
    print(syn6.read(8), syn6.read(4).hex(),syn6.read(2).hex())
relocation_table = syn6.read()

print("Relocation Fix Up Information")
print(hex(len(relocation_table)))


Magic number 601a Alcyon-Format Object Module
Text Size 00002178
Data Size 00000000
BSS Size 00000000
Symbol Size 00001d34
Reserved (must be zero) 00000000000000000000
Symbol Table
b'DISABLET' 88000000 0000
b'ENABLETI' 88000000 0000
b'INITDSP\x00' a2000000 0026
b'COPYLOOP' 82000000 0056
b'MAKESAMV' 82000000 00b4
b'BG\x00\x00\x00\x00\x00\x00' c00000f0 0058
b'HC\x00\x00\x00\x00\x00\x00' c00000f0 0004
b'HP\x00\x00\x00\x00\x00\x00' c00000f0 002e
b'HS\x00\x00\x00\x00\x00\x00' c00000f0 0034
b'VC\x00\x00\x00\x00\x00\x00' c00000f0 0006
b'VI\x00\x00\x00\x00\x00\x00' c00000f0 004e
b'VP\x00\x00\x00\x00\x00\x00' c00000f0 003e
b'VS\x00\x00\x00\x00\x00\x00' c00000f0 0044
b'HBB\x00\x00\x00\x00\x00' c00000f0 0030
b'HBE\x00\x00\x00\x00\x00' c00000f0 0032
b'HDE\x00\x00\x00\x00\x00' c00000f0 003c
b'HEQ\x00\x00\x00\x00\x00' c00000f0 0054
b'HVS\x00\x00\x00\x00\x00' c00000f0 0036
b'LPH\x00\x00\x00\x00\x00' c00000f0 0008
b'LPV\x00\x00\x00\x00\x00' c00000f0 000a
b'OB0\x00\x00\x00\x00\x00' c00000f0 0010
b'OB1\

In [52]:
open("syn6.bin",'wb').write(text_data)


8568

Now go into IDA Pro, disassemble (using M68000) and export as syn6.s

[do stuff in ida pro]

Now let's rebuild using rmac. Download and build rmac:

Rebuild syn6.s using rmac and find the differences:

In [160]:
!./rmac/rmac -s ~oall -n -l*syn6prn -ps -i syn6.s -o syn6
!diff -y -W 120 <(dd if=syn6|xxd) <(dd if=syn6.o|xxd)

Legacy mode OFF
syn6.s 84: Warning: unoptimized short branch
syn6.s 1022: Warning: unoptimized short branch
27+1 records in
27+1 records out
48+1 records in
48+1 records out
24640 bytes (25 kB, 24 KiB) copied, 5.3004e-05 s, 465 MB/s
14279 bytes (14 kB, 14 KiB) copied, 4.7309e-05 s, 302 MB/s
00000000: 601a 0000 2178 0000 0000 0000 0000 0000  `...!	00000000: 601a 0000 2178 0000 0000 0000 0000 0000  `...!
00000010: 1626 0000 0000 0000 0000 0000 436f 7079  .&...   |	00000010: 1d34 0000 0000 0000 0000 0000 436f 7079  .4...
00000020: 7269 6768 7420 2831 3939 3329 2049 6d61  right	00000020: 7269 6768 7420 2831 3939 3329 2049 6d61  right
00000030: 6769 7465 6320 4465 7369 676e 2c20 496e  gitec	00000030: 6769 7465 6320 4465 7369 676e 2c20 496e  gitec
00000040: 6320 48e7 fffe 23fc 0000 0000 00f1 a114  c H..	00000040: 6320 48e7 fffe 23fc 0000 0000 00f1 a114  c H..
00000050: 23fc 0000 0000 00f1 a100 227c 0000 0000  #....	00000050: 23fc 0000 0000 00f1 a100 227c 0000 0000  #....
00000060: 207c 0000 

In [95]:
#!cat syn6prn

In [118]:
def get_symbol_table(fname):
    syn6 = open(fname,'rb')
    syn6.seek(0)
    magic_number = syn6.read(2)
    text_size = syn6.read(4)
    data_size = syn6.read(4)
    bss_size = syn6.read(4).hex()
    sym_size = syn6.read(4)
    print("Symbol Size",sym_size.hex())
    reserved = syn6.read(10)
    text_data = syn6.read(int.from_bytes(text_size, "big"))

    symbol_table = []
    sym_size_int = int.from_bytes(sym_size, "big")
    for i in range(0,int(sym_size_int/14)):
        symbol = syn6.read(8).decode('utf-8').replace('\x00','')
        info = syn6.read(2).hex()
        sym_address = syn6.read(4).hex()
        symbol_table += [(sym_address,info,symbol)]

    return sorted(symbol_table)

symbol_table = set(get_symbol_table("syn6.o"))
my_symbol_table = get_symbol_table("syn6")


Symbol Size 00001d34
Symbol Size 00001084


In [108]:
set([x[1] for x in symbol_table])

{'8200', '8800', 'a200', 'c000', 'e000'}

In [143]:
print('\n'.join(sorted(set(sorted([f"{x[2]} EQU ${x[0]}" 
                            for x in symbol_table 
                            ])))))


A1_BASE EQU $00f02200
A1_CLIP EQU $00f02208
A1_FINC EQU $00f02220
A1_FLAGS EQU $00f02204
A1_FPIXE EQU $00f02218
A1_FSTEP EQU $00f02214
A1_INC EQU $00f0221c
A1_PIXEL EQU $00f0220c
A1_STEP EQU $00f02210
A2_BASE EQU $00f02224
A2_FLAGS EQU $00f02228
A2_MASK EQU $00f0222c
A2_PIXEL EQU $00f02230
A2_STEP EQU $00f02234
BASE EQU $00f00000
BG EQU $00f00058
BORD1 EQU $00f0002a
BORD2 EQU $00f0002c
B_CMD EQU $00f02238
B_COUNT EQU $00f0223c
B_DSTD EQU $00f02248
B_DSTZ EQU $00f02250
B_I0 EQU $00f02288
B_I1 EQU $00f02284
B_I2 EQU $00f02280
B_I3 EQU $00f0227c
B_IINC EQU $00f02270
B_PATD EQU $00f02268
B_SRCD EQU $00f02240
B_SRCZ1 EQU $00f02258
B_SRCZ2 EQU $00f02260
B_STOP EQU $00f02278
B_Z0 EQU $00f02298
B_Z1 EQU $00f02294
B_Z2 EQU $00f02290
B_Z3 EQU $00f0228c
B_ZINC EQU $00f02274
CHANFLAG EQU $00f1b358
CHANGEFX EQU $000003b0
CHANGE_V EQU $0000042a
CHRO_CLK EQU $00f10014
CLK1 EQU $00f10010
CLK2 EQU $00f10012
CLUT EQU $00f00400
COPYLOOP EQU $00000056
DISABLET EQU $00000000
DISABLE_ EQU $000005a0
DSPORG E

In [140]:
#define	AL_DEFINED	0x8000
#define	AL_EQUATED	0x4000
#define	AL_GLOBAL	0x2000
#define	AL_EQUREG	0x1000
#define	AL_EXTERN	0x0800
#define	AL_DATA		0x0400
#define	AL_TEXT		0x0200
#define	AL_BSS		0x0100
#define	AL_FILE		0x0080

my_symbols = [x[2] for x in my_symbol_table]
print('\n'.join(sorted(set(sorted([f"{x[2]} EQU ${x[0]}" 
                            for x in symbol_table 
                            if x[1]=='e000' and x[2] not in my_symbols])))))


CHANFLAG EQU $00f1b358
DSPORG EQU $00f1b000
FXONLY EQU $00f1b35c
SEMAPHOR EQU $80000080


In [70]:
#TODO: How do we add the relocation table?

## Scratchpad

In [67]:
open("test.s",'w').write("ori.b #$FF,d0")
!./rmac/rmac -s ~oall -n -l*testprn -ps -i test.s -o test
!cat testprn

Legacy mode OFF
    1  00000000  000000FF               ori.b #$FF,d0



In [96]:
open("test.s",'w').write("""
NDI EQU $FF00
.text
lea     $1DE0,a2
byte1: dc.b $00
""")
!./rmac/rmac -s ~oall -n -l*testprn -ps -i test.s -o test
!dd if=test|xxd 2> /dev/null
!cat testprn

Legacy mode OFF
0+1 records in
0+1 records out
68 bytes copied, 2.4071e-05 s, 2.8 MB/s
00000000: 601a 0000 0008 0000 0000 0000 0000 0000  `...............
00000010: 001c 0000 0000 0000 0000 0000 45f9 0000  ............E...
00000020: 1de0 0000 4e44 4900 0000 0000 c000 0000  ....NDI.........
00000030: ff00 6279 7465 3100 0000 8200 0000 0006  ..byte1.........
00000040: 0000 0000                                ....
    1                                   
    2           =0000FF00               NDI EQU $FF00
    3                                   .text
    4  00000000  45F900001DE0           lea     $1DE0,a2
    5  00000006  00                     byte1: dc.b $00

               NDI 000000000000FF00  a 
             byte1 0000000000000006  t 



In [98]:
labels = """init_sound
nt_vbl
pt_mod_init
start_mod
stop_mod
playfx2
change_volume
set_volume
nofade
fadeup
fadedown
enable_fx
disable_fx
disable_fx
disable_fx
resume_dsp""".split('\n')
syn6 = open("moomoo.bin",'rb')
syn6.seek(0)
for l in labels:
    _, a = syn6.read(2).hex(),syn6.read(4).hex()
    print(f"{l} EQU ${a}")


init_sound EQU $000041ba
nt_vbl EQU $0000425e
pt_mod_init EQU $000041d6
start_mod EQU $000045e2
stop_mod EQU $00004606
playfx2 EQU $00004370
change_volume EQU $000044ea
set_volume EQU $00004546
nofade EQU $00004598
fadeup EQU $000045b2
fadedown EQU $000045ca
enable_fx EQU $0000464e
disable_fx EQU $00004660
disable_fx EQU $00004660
disable_fx EQU $00004660
resume_dsp EQU $00004470
